                                           **K-means Clustering** 

---



# Part 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import cv2

%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

from google.colab.patches import cv2_imshow
from keras.datasets import cifar10
from sklearn.metrics import silhouette_score
! pip install validclust
from validclust import dunn
from sklearn.metrics import pairwise_distances


In [ ]:
# Load cifar-10 data-set

(trainX, trainy), (testX, testy) = cifar10.load_data()

In [ ]:
#Convert the images to grayscale using cv2.
#Normalize the images.
#Reshape the images from (10000,32,32) to (10000,1024).



# transform data

testX= np.array([cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) for image in testX])

testX=testX/255






In [ ]:
#Kmeans Clustering algorithm

# Step perfromed in the algorithm:-
"""
1) take ramdom points from the data of length equal to the no of clusters as the initial centroids of the clusters
2)find the closest centroid for each point and asign it to them
3) take the mean as each cluster as the new centroid 
4) repeat step 2 and 3 until the difference between new and old centroid is equal to zero or epochs complete
"""
#function for euclidean distance between two data points 
def euclidean_distance(d1, d2):
    return np.sqrt(np.sum((d1 - d2) ** 2))


class KmeansClustering:
    def __init__(self, no_of_cluster, epochs):
        self.no_of_cluster = no_of_cluster
        self.epochs = epochs
        
        # points in a  cluster
        self.clusters=[]
        for _ in range(self.no_of_cluster):
          self.clusters.append([]) 
        # Cluster Centers
        self.centroids = []

    def predict(self, X):
        self.X = X
        self.no_of_points, self.no_of_features = X.shape

        # initializing cluster centroids randomly
        random_points = np.random.choice(self.no_of_points, self.no_of_cluster, replace=False)
        for i in random_points:
          self.centroids.append(self.X[i]) 

        # creating clusters by asigning point closest to the cluster centroid to that clustser
        for _ in range(self.epochs):
            
            self.clusters = self.cluster_create(self.centroids)
           
            # update centorids from the clusters
            old_centroids = self.centroids
            # update centroids to mean of the points in the cluster
            centroids = np.zeros((self.no_of_cluster, self.no_of_features))
            for i, cluster in enumerate(self.clusters):
              new_centroid = np.mean(self.X[cluster], axis=0)
              centroids[i] = new_centroid
            self.centroids=centroids
             
            # check if clusters have changed(old-centroid=new-centroid)
            distances = []
            for i in range(self.no_of_cluster):
              distances.append(euclidean_distance(old_centroids[i], self.centroids[i])) 
        
       
    
            if sum(distances)==0:
              print("Cluster centroids have converged")
              break
          
        # Classify samples as the index of their clusters
        labels = np.empty(self.no_of_points)

        for i, cluster in enumerate(self.clusters):
            for j in cluster:
                labels[j] = i
        return labels



    def cluster_create(self, centroids):
        # Assign points to the closest centroid
        clusters=[]
        for _ in range(self.no_of_cluster):
          clusters.append([])
        
        for i, point in enumerate(self.X):
          distances=[]
          for centroid in centroids:
            distances.append(euclidean_distance(point, centroid)) 
          closest_index = np.argmin(distances)
          clusters[closest_index].append(i)
        return clusters
    





In [ ]:
model=KmeansClustering(no_of_cluster=10,epochs=100)


In [ ]:
labels=model.predict(testX)

Cluster centroids have converged


In [ ]:
# perform ASC and dunn score analysis

dist= pairwise_distances(testX)
print("Dunn score is: ",dunn(dist,labels))
print("ASC score is: ",silhouette_score(testX,labels))

Dunn score is:  0.0902795391612921
ASC score is:  0.05360154438360176


# Part 2

Steps followed:

1. use autoencoders to fit data
2. use the encoder part to generate a sparse dataset
3. use k-means to generate the labels on the sparse dataset and perform asc to 

In [ ]:
#loading libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

from tensorflow import keras
from sklearn.metrics import silhouette_score

from keras.layers import Dense, Input
from tensorflow.keras.optimizers import  Adam
from keras.models import Model
from keras.datasets import cifar10



In [ ]:
#loading cifar-10 dataset
(train_x,train_y),(test_x,test_y)= cifar10.load_data()

In [ ]:
# transform data

train_x= np.array([cv2.cvtColor(image,cv2.COLOR_BGR2GRAY) for image in train_x])
train_x = train_x.astype('float32') / 255.
train_x = train_x.reshape(50000, 1024)

print(train_x.shape)

(50000, 1024)


In [ ]:
#encoder 
input_img = keras.Input(shape=(1024,))
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)

#decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(1024, activation='sigmoid')(decoded)

In [ ]:
#autoencoder model
autoencoder = keras.Model(input_img, decoded)

#encoder model
encoder = Model(input_img, encoded)


In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

#fitting autoencoder
autoencoder.fit(train_x, train_x,
                epochs=100,
                batch_size=128,
                shuffle=True,
                )

sparse_x = encoder.predict(train_x)



Epoch 1/100
391/391 [==============================] - 6s 13ms/step - loss: 0.6268
Epoch 2/100
391/391 [==============================] - 5s 13ms/step - loss: 0.6022
Epoch 3/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5972
Epoch 4/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5955
Epoch 5/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5941
Epoch 6/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5925
Epoch 7/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5916
Epoch 8/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5908
Epoch 9/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5899
Epoch 10/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5892
Epoch 11/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5888
Epoch 12/100
391/391 [==============================] - 5s 13ms/step - loss: 0.5882
E

Performing K-Means on the sparse dataset recieeved

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
#fitting kmeans on the sparse_x
model = KMeans(n_clusters=10, n_init=1, max_iter=1)
model.fit(sparse_x)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=1,
       n_clusters=10, n_init=1, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [ ]:
#Asc score of the learnt model
print("ASC score is: ",silhouette_score(sparse_x,model.labels_))

ASC score is:  0.06722549


In [ ]:
import cv2

TypeError: ignored

(1024,)